<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/Genrespotifyclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np

df_2000 = pd.read_csv("/content/Spotify-2000.csv")

In [15]:

# check data types using df.dtype
df_2000["Top Genre"].value_counts()

df_2000.drop(columns = ['Index', 'Title', 'Artist', 'Year'], inplace = True)

In [16]:
len(df_2000["Top Genre"].unique())
attributes = df_2000.columns[1:]
for attribute in attributes:
    temp = df_2000[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df_2000[attribute][instance] = float(temp[instance].replace(',',''))


<ipython-input-16-a2f61fe28107>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000[attribute][instance] = float(temp[instance].replace(',',''))


In [17]:
df = df_2000

# first extracting the genre columns
# getting rid of white spaces and turning it all into lower cases
genre = (df["Top Genre"].str.strip()).str.lower()

# function to split the genre column
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ")
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

In [19]:
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ")).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)


In [20]:
len(genre_m1.unique())

genre_m1.value_counts()

unique = genre_m1.unique()
to_remove = []

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 10: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

51

In [21]:
df['Top Genre'] = genre_m1
df

,Top Genre,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,standards,157,30,53,-14,11,68,201.0,94,3,71
1,rock,135,79,50,-11,17,81,207.0,17,7,39
2,hip,168,69,66,-9,7,52,341.0,2,17,69
3,metal,173,96,43,-4,3,37,269.0,0,4,76
4,rock,106,82,58,-5,10,87,256.0,1,3,59
...,...,...,...,...,...,...,...,...,...,...,...
1989,standards,94,21,70,-12,11,72,128.0,84,7,63
1990,standards,175,76,36,-8,76,95,136.0,73,6,69
1991,rock,168,80,53,-9,31,97,162.0,74,7,74
1992,bebop,174,26,45,-13,7,60,324.0,54,4,65


In [22]:
df.set_index(["Top Genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)

df["Top Genre"].value_counts()

rock         834
pop          307
standards    123
metal         93
indie         78
cabaret       51
soul          45
wave          40
invasion      36
europop       27
folk          26
hip           23
uk            22
disco         18
funk          18
mellow        17
country       17
dance         16
americana     13
britpop       11
gold          11
beat          11
limburg       10
Name: Top Genre, dtype: int64

In [23]:
df.dtypes

Top Genre                 object
Beats Per Minute (BPM)     int64
Energy                     int64
Danceability               int64
Loudness (dB)              int64
Liveness                   int64
Valence                    int64
Length (Duration)         object
Acousticness               int64
Speechiness                int64
Popularity                 int64
dtype: object

In [24]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
# training set
X_train = train_set.values[:,1:]
y_train = train_set.values[:,0]

# test set
X_test = test_set.values[:,1:]
y_test = test_set.values[:,0]

from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

# obtaining all unique classes
unique = np.unique(y_train)

In [25]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
# 1 hot encoding
y_test_1hot = label_binarize(y_test, classes = unique)
y_train_1hot = label_binarize(y_train, classes = unique)

# labelling
y_test_label = LabelEncoder()


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier

In [28]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

In [34]:
# training the models
model_method1 = LogisticRegression(multi_class = 'ovr').fit(X_train_ST, y_train)

# getting predictions
predictions_method1 = model_method1.predict(X_test_ST)

model = RandomForestClassifier(random_state = 42, min_samples_split = 5)
model.fit(X_train, y_train)
pred = model.predict(X_test_ST)

accuracy = accuracy_score(y_test, predictions_method1)
print(accuracy)
from sklearn.metrics import confusion_matrix
print(f1_score(y_test, predictions_method1, labels = unique, average = 'micro' ))


0.4756756756756757
0.4756756756756757
